In [9]:
!pip install tensorflow keras 

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [16]:
import os
import shutil
import random

# Define paths
data_dir = 'data/'
train_dir = 'train/'
validation_dir = 'validation/'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Get list of image filenames in each class folder
malignant_images = os.listdir(os.path.join(data_dir, 'malignant'))
benign_images = os.listdir(os.path.join(data_dir, 'normal'))

# Shuffle the filenames
random.shuffle(malignant_images)
random.shuffle(benign_images)

# Split ratio (adjust as needed)
split_ratio = 0.8  # 80% for training, 20% for validation

# Calculate split index
malignant_split_idx = int(len(malignant_images) * split_ratio)
benign_split_idx = int(len(benign_images) * split_ratio)

# Move images to train and validation folders
for img in malignant_images[:malignant_split_idx]:
    src = os.path.join(data_dir, 'malignant', img)
    dst = os.path.join(train_dir, 'malignant', img)
    shutil.move(src, dst)

for img in malignant_images[malignant_split_idx:]:
    src = os.path.join(data_dir, 'malignant', img)
    dst = os.path.join(validation_dir, 'malignant', img)
    shutil.move(src, dst)

for img in benign_images[:benign_split_idx]:
    src = os.path.join(data_dir, 'normal', img)
    dst = os.path.join(train_dir, 'normal', img)
    shutil.move(src, dst)

for img in benign_images[benign_split_idx:]:
    src = os.path.join(data_dir, 'normal', img)
    dst = os.path.join(validation_dir, 'normal', img)
    shutil.move(src, dst)


In [17]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'train/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')  # or categorical if more than two classes

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
        'validation/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')  # or categorical if more than two classes


Found 548 images belonging to 2 classes.
Found 139 images belonging to 2 classes.


In [18]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # or softmax if more than two classes
])

model.summary()


C:\Users\maham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # or categorical_crossentropy if more than two classes
              metrics=['accuracy'])


In [20]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # total_train // batch_size
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # total_validation // batch_size
      verbose=2)


Epoch 1/15


C:\Users\maham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 - 28s - 277ms/step - accuracy: 0.6989 - loss: 0.4862 - val_accuracy: 0.6115 - val_loss: 0.4265
Epoch 2/15
100/100 - 8s - 83ms/step - accuracy: 0.7755 - loss: 0.4115 - val_accuracy: 0.7410 - val_loss: 0.3859
Epoch 3/15
100/100 - 9s - 88ms/step - accuracy: 0.7901 - loss: 0.4182 - val_accuracy: 0.7986 - val_loss: 0.3625
Epoch 4/15
100/100 - 9s - 86ms/step - accuracy: 0.8358 - loss: 0.4044 - val_accuracy: 0.6475 - val_loss: 0.3611
Epoch 5/15
100/100 - 9s - 86ms/step - accuracy: 0.8668 - loss: 0.3847 - val_accuracy: 0.7194 - val_loss: 0.5623
Epoch 6/15
100/100 - 9s - 86ms/step - accuracy: 0.8613 - loss: 0.3026 - val_accuracy: 0.8201 - val_loss: 0.3600
Epoch 7/15
100/100 - 8s - 82ms/step - accuracy: 0.8978 - loss: 0.2209 - val_accuracy: 0.8489 - val_loss: 0.3137
Epoch 8/15
100/100 - 8s - 80ms/step - accuracy: 0.9234 - loss: 0.1566 - val_accuracy: 0.8273 - val_loss: 0.5067
Epoch 9/15
100/100 - 9s - 85ms/step - accuracy: 0.8978 - loss: 0.2145 - val_accuracy: 0.8345 - val_loss: 0.2861
E

In [21]:
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step - accuracy: 0.8507 - loss: 0.2774
Validation Loss: 0.2782706022262573
Validation Accuracy: 0.8633093237876892
